In [3]:
# pip install selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from tqdm.auto import tqdm 
from concurrent.futures import ThreadPoolExecutor
import time
import re
import pandas as pd


c:\Users\piano\anaconda3\envs\hyojung_2015479\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
artist_links = pd.read_csv('moma_artists_links.csv')
artist_links.head()

,artist_link
0,https://artfacts.net/artist/elinor-bellingham-...
1,https://artfacts.net/artist/frank-brangwyn/30490
2,https://artfacts.net/artist/stephen-conroy/1625
3,https://artfacts.net/artist/hans-feibusch/30758
4,https://artfacts.net/artist/james-flewitt-mull...


In [5]:
artist_links = artist_links.dropna()
artist_links.shape

(47717, 1)

In [6]:
artist_links["artist_link"] = artist_links["artist_link"].apply(lambda x: x + "/exhibitions")

exhibitions = artist_links.copy()

In [7]:
hrefs = exhibitions['artist_link'].dropna()
hrefs

0        https://artfacts.net/artist/elinor-bellingham-...
1        https://artfacts.net/artist/frank-brangwyn/304...
2        https://artfacts.net/artist/stephen-conroy/162...
3        https://artfacts.net/artist/hans-feibusch/3075...
4        https://artfacts.net/artist/james-flewitt-mull...
                               ...                        
47712    https://artfacts.net/artist/alyson-shotz/16984...
47713    https://artfacts.net/artist/tatjana-valsang/30...
47714    https://artfacts.net/artist/knutte-wester/2591...
47715    https://artfacts.net/artist/katrin-westman/394...
47716    https://artfacts.net/artist/martin-wickstroem/...
Name: artist_link, Length: 47717, dtype: object

In [10]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
#  chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--disable-dev-shm-usage')

# 올바른 ChromeDriver 경로 지정
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()


# 대상 웹 페이지 URL
target_url = 'https://artfacts.net/artist/gunnar-aagaard-andersen/27157/exhibitions'
driver.get(target_url)

##### 로그인 #####

try:
    # 페이지가 완전히 로드될 때까지 기다림
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//a[text()='Login']")))
    
    # 로그인 옵션 클릭
    login_option = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[text()='Login']")))
    login_option.click()
except TimeoutException:
    print("Login button not found")
    driver.quit()

try:
    # 로그인 세부사항 입력 및 제출
    email_field = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "email")))
    email_field.send_keys('wkyoung2@gmail.com')
    
    password_field = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "password")))
    password_field.send_keys('!BAwk123123')
except TimeoutException:
    print("Login form elements not found")
    driver.quit()

try:
    # login 버튼 선택 및 클릭
    login_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "app-js-components-AuthenticationModal-AuthenticationModal__button"))
    )
    login_button.click()
except TimeoutException:
    print("Login button not clickable")
    driver.quit()

# 로그인 후 추가 대기 시간
time.sleep(5)


In [17]:
def collect_museum_collections_info():
    try:
        section_header = driver.find_element(By.XPATH, "//h3[text()='Museum Collections']")
        artist_card_list = section_header.find_element(By.XPATH, "./ancestor::div[contains(@class, 'app-js-components-ArtistCard-ArtistCard__list')]")
        section_containers = artist_card_list.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__container')]")

        all_items = []
        pending_items = []

        for container in section_containers:
            labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")

            for element in labels_and_items:
                if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
                    current_country = element.text.strip()
                    
                    # Assign the current label to all pending items
                    for item in pending_items:
                        item['Country'] = current_country
                        all_items.append(item)
                    pending_items.clear()
                elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
                    li_elements = element.find_elements(By.TAG_NAME, "li")

                    for li_element in li_elements:
                        span_elements = li_element.find_elements(By.TAG_NAME, "span")
                        link = li_element.find_element(By.TAG_NAME, "a").get_attribute('href')

                        if len(span_elements) >= 2:
                            name = span_elements[0].text.strip()
                            city = span_elements[1].text.strip()
                        elif len(span_elements) == 1:
                            name = span_elements[0].text.strip()
                            city = 'N/A'
                        else:
                            name = li_element.text.strip()
                            city = 'N/A'

                        pending_items.append({
                            'Country': None,  # Placeholder for later label assignment
                            'Name': name,
                            'City': city,
                            'url': link
                        })

        # Assign the last known label to any remaining pending items
        for item in pending_items:
            item['Country'] = current_country
            all_items.append(item)

        return all_items
    except Exception as e:
        return []

def collect_section_info(section_title):
    try:
        section = driver.find_element(By.XPATH, f"//div[h3[text()='{section_title}']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")
        
        all_items = []
        pending_items = []
        current_year = None
        
        for container in items_container:
            labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")
            
            for element in labels_and_items:
                if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
                    current_year = element.text.strip()
                    
                    # Add pending items with the current year
                    for item in pending_items:
                        item['Year'] = current_year
                        all_items.append(item)
                    pending_items.clear()
                    
                elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
                    li_elements = element.find_elements(By.TAG_NAME, "li")
                    for li_element in li_elements:
                        a_tags = li_element.find_elements(By.TAG_NAME, "a")
                        if len(a_tags) >= 4:
                            try:
                                name = a_tags[0].get_attribute('innerText')
                            except:
                                name = a_tags[0].text
                            try:
                                institution = a_tags[1].get_attribute('innerText')
                            except:
                                institution = a_tags[1].text
                            try:
                                city = a_tags[2].get_attribute('innerText')
                            except:
                                city = a_tags[2].text
                            try:
                                country = a_tags[3].get_attribute('innerText')
                            except:
                                country = a_tags[3].text
                            pending_items.append({
                                'Year': current_year,  # Will be replaced when the next label is found
                                'Name': name,
                                'Institution': institution,
                                'City': city,
                                'Country': country,
                                'url': a_tags[0].get_attribute('href')
                            })
        
        # Add any remaining pending items
        for item in pending_items:
            item['Year'] = current_year
            all_items.append(item)
        
        return all_items
    except Exception as e:
        return None

def collect_dealer_directory_info():
    try:
        section = driver.find_element(By.XPATH, "//div[h3[text()='Dealer Directory']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")

        all_items = []
        for container in items_container:
            labels = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__label")
            items = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__item")
            for label, item in zip(labels, items):
                all_items.append({
                    'Country': label.text,
                    'Name': item.text,
                    'url': item.find_element(By.TAG_NAME, "a").get_attribute('href')
                })
        return all_items
    except Exception as e:
        return []

def collect_catalogs_info():
    try:
        section = driver.find_element(By.XPATH, "//div[h3[text()='Catalogs']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")

        all_items = []
        for container in items_container:
            labels = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__label")
            items = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__item")
            for label, item in zip(labels, items):
                all_items.append({
                    'Year': label.text,
                    'Name': item.text,
                    'url': item.find_element(By.TAG_NAME, "a").get_attribute('href')
                })
        return all_items
    except Exception as e:
        return []
    

def collect_exhibitions_info(section_title):
    try:
        section = driver.find_element(By.XPATH, f"//div[h3[text()='{section_title}']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")
        
        all_items = []
        pending_items = []
        current_year = None
        
        for container in items_container:
            labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")
            
            for element in labels_and_items:
                if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
                    current_year = element.text.strip()
                    
                    # Add pending items with the current year
                    for item in pending_items:
                        item['Year'] = current_year
                        all_items.append(item)
                    pending_items.clear()
                    
                elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
                    li_elements = element.find_elements(By.TAG_NAME, "li")
                    for li_element in li_elements:
                        a_tags = li_element.find_elements(By.TAG_NAME, "a")
                        if len(a_tags) >= 4:
                            try:
                                name = a_tags[0].get_attribute('innerText')
                            except:
                                name = a_tags[0].text
                            try:
                                institution = a_tags[1].get_attribute('innerText')
                            except:
                                institution = a_tags[1].text
                            try:
                                city = a_tags[2].get_attribute('innerText')
                            except:
                                city = a_tags[2].text
                            try:
                                country = a_tags[3].get_attribute('innerText')
                            except:
                                country = a_tags[3].text

                            # Group Exhibition Type
                            exhibition_type = 'Group Exhibition' if 'Group Exhibition' in li_element.text else 'Solo Exhibition'

                            pending_items.append({
                                'Year': current_year,  # Will be replaced when the next label is found
                                'Name': name,
                                'Institution': institution,
                                'City': city,
                                'Country': country,
                                'Type': exhibition_type,  # Group or Solo Exhibition
                                'url': a_tags[0].get_attribute('href')
                            })
        
        # Add any remaining pending items
        for item in pending_items:
            item['Year'] = current_year
            all_items.append(item)
        
        return all_items
    except Exception as e:
        return None



In [21]:
all_info = []

for href in tqdm(hrefs):
    match = re.search(r'https://artfacts\.net/(?:institution|artist)/([^/]+)/\d+/(?:artists|exhibitions)', href)
    label = match.group(1) if match else None

    driver.get(href)
    time.sleep(2)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    try:
        call_to_action_icons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "app-js-components-CallToAction-CallToAction__iconLink"))
        )
    except Exception:
        call_to_action_icons = []

    for call_to_action_icon in call_to_action_icons:
        try:
            driver.execute_script("arguments[0].click();", call_to_action_icon)
            time.sleep(0.5)
        except Exception:
            pass

    time.sleep(1.5)

    exhibitions = collect_exhibitions_info('Exhibitions')
    biennials = collect_section_info('Biennials')
    art_fairs = collect_section_info('Art fairs')
    museum_collections = collect_museum_collections_info()
    dealer_directory = collect_dealer_directory_info()
    catalogs = collect_catalogs_info()

    if exhibitions:
        for exhibition in exhibitions:
            all_info.append({'Artist': label, **exhibition})

    if biennials:
        for exhibition in biennials:
            all_info.append({'Artist': label, 'Type': 'Biennials', **exhibition})

    if art_fairs:
        for fair in art_fairs:
            if fair:
                all_info.append({'Artist': label, 'Type': 'Art Fairs', **fair})

    if museum_collections:
        for collection in museum_collections:
            if collection:
                all_info.append({'Artist': label, 'Type': 'Museum Collections', **collection})

    if dealer_directory:
        for dealer in dealer_directory:
            if dealer:
                all_info.append({'Artist': label, 'Type': 'Dealer Directory', **dealer})

    if catalogs:
        for catalog in catalogs:
            if catalog:
                all_info.append({'Artist': label, 'Type': 'Catalogs', **catalog})

    time.sleep(1)
    print(all_info)


  0%|          | 1/47717 [00:17<236:39:33, 17.86s/it]

[{'Artist': 'elinor-bellingham-smith', 'Year': '2010', 'Name': 'Unpopular Culture', 'Institution': 'Mead Gallery - Warwick Arts Centre', 'City': 'Coventry', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/unpopular-culture/506913'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1999', 'Name': 'Suffolk : A Female Focus', 'Institution': 'Wolsey Art Gallery & The Room Upstairs', 'City': 'Ipswich', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/suffolk-:-a-female-focus/324841'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1954', 'Name': 'Recent British Drawings', 'Institution': 'Institute of Contemporary Arts London (ICA)', 'City': 'London', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/recent-british-drawings/521672'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1952', 'Name': 'Recent Trends in Realist Painting', 'Institution': 'Insti

  0%|          | 2/47717 [00:29<191:52:52, 14.48s/it]

[{'Artist': 'elinor-bellingham-smith', 'Year': '2010', 'Name': 'Unpopular Culture', 'Institution': 'Mead Gallery - Warwick Arts Centre', 'City': 'Coventry', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/unpopular-culture/506913'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1999', 'Name': 'Suffolk : A Female Focus', 'Institution': 'Wolsey Art Gallery & The Room Upstairs', 'City': 'Ipswich', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/suffolk-:-a-female-focus/324841'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1954', 'Name': 'Recent British Drawings', 'Institution': 'Institute of Contemporary Arts London (ICA)', 'City': 'London', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/recent-british-drawings/521672'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1952', 'Name': 'Recent Trends in Realist Painting', 'Institution': 'Insti

  0%|          | 3/47717 [00:42<180:36:14, 13.63s/it]

...

[{'Artist': 'elinor-bellingham-smith', 'Year': '2010', 'Name': 'Unpopular Culture', 'Institution': 'Mead Gallery - Warwick Arts Centre', 'City': 'Coventry', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/unpopular-culture/506913'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1999', 'Name': 'Suffolk : A Female Focus', 'Institution': 'Wolsey Art Gallery & The Room Upstairs', 'City': 'Ipswich', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/suffolk-:-a-female-focus/324841'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1954', 'Name': 'Recent British Drawings', 'Institution': 'Institute of Contemporary Arts London (ICA)', 'City': 'London', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/recent-british-drawings/521672'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1952', 'Name': 'Recent Trends in Realist Painting', 'Institution': 'Insti

  0%|          | 7/47717 [01:24<143:35:31, 10.83s/it]

[{'Artist': 'elinor-bellingham-smith', 'Year': '2010', 'Name': 'Unpopular Culture', 'Institution': 'Mead Gallery - Warwick Arts Centre', 'City': 'Coventry', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/unpopular-culture/506913'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1999', 'Name': 'Suffolk : A Female Focus', 'Institution': 'Wolsey Art Gallery & The Room Upstairs', 'City': 'Ipswich', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/suffolk-:-a-female-focus/324841'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1954', 'Name': 'Recent British Drawings', 'Institution': 'Institute of Contemporary Arts London (ICA)', 'City': 'London', 'Country': 'United Kingdom', 'Type': 'Group Exhibition', 'url': 'https://artfacts.net/exhibition/recent-british-drawings/521672'}, {'Artist': 'elinor-bellingham-smith', 'Year': '1952', 'Name': 'Recent Trends in Realist Painting', 'Institution': 'Insti

  0%|          | 7/47717 [01:33<177:35:20, 13.40s/it]


KeyboardInterrupt: 

In [22]:
all_info

[{'Artist': 'elinor-bellingham-smith',
  'Year': '2010',
  'Name': 'Unpopular Culture',
  'Institution': 'Mead Gallery - Warwick Arts Centre',
  'City': 'Coventry',
  'Country': 'United Kingdom',
  'Type': 'Group Exhibition',
  'url': 'https://artfacts.net/exhibition/unpopular-culture/506913'},
 {'Artist': 'elinor-bellingham-smith',
  'Year': '1999',
  'Name': 'Suffolk : A Female Focus',
  'Institution': 'Wolsey Art Gallery & The Room Upstairs',
  'City': 'Ipswich',
  'Country': 'United Kingdom',
  'Type': 'Group Exhibition',
  'url': 'https://artfacts.net/exhibition/suffolk-:-a-female-focus/324841'},
 {'Artist': 'elinor-bellingham-smith',
  'Year': '1954',
  'Name': 'Recent British Drawings',
  'Institution': 'Institute of Contemporary Arts London (ICA)',
  'City': 'London',
  'Country': 'United Kingdom',
  'Type': 'Group Exhibition',
  'url': 'https://artfacts.net/exhibition/recent-british-drawings/521672'},
 {'Artist': 'elinor-bellingham-smith',
  'Year': '1952',
  'Name': 'Recent T

In [25]:
# 수집한 모든 정보를 DataFrame으로 변환
df = pd.DataFrame(all_info)
df
df.to_excel('artists_exhibitions_info.xlsx', index=False)